## Import & Install

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#!pip install mljar-supervised

In [ ]:
import pandas as pd
import random
import os
import numpy as np
import warnings
import matplotlib.pylab as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from datetime import datetime

from sklearn.preprocessing import RobustScaler, PowerTransformer

from tqdm import tqdm

from supervised.automl import AutoML

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


## Fix Seed

In [ ]:
#Seed 고정
class CFG:
    SEED = 42

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

## Data Load

In [ ]:
###  INPUT ###
input_data = pd.read_csv('/content/drive/MyDrive/Smart/2023_smartFarm_AI_hackathon_dataset.csv')

## Data Pre-processing

In [ ]:
input_data = input_data.drop(columns=['frmDist'])

# Split the data into training and testing sets
X = input_data[input_data.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = input_data[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]

In [ ]:
X.columns = X.columns.str.replace(' ', '_')

In [ ]:
value_counts_per_column = X.nunique()
multi_value_columns = value_counts_per_column[value_counts_per_column > 1].index
X = X[multi_value_columns]

In [ ]:
X = X.drop(columns=['frmYear', 'frmWeek'])

## Scaling

In [ ]:
num_features = X.columns[X.columns != 'date']

In [ ]:
scaler = PowerTransformer()
X[num_features] = scaler.fit_transform(X[num_features])

## Feature Selection

In [ ]:
X_1 = X.drop(columns=['tcdmt',
                      'WaterUsage',
                      'WaterCost',
                      'FertilizerUsage',
                      'FertilizerCost',
                      'CO2Usage',
                      'CO2Cost',
                      'MistUsageTime',
                      'Mist_Cost'])

In [ ]:
X_2 = X.drop(columns=['hvstCo'])

## Add time variable

In [ ]:
date = pd.to_datetime(X_1['date'], format='%Y%m%d')
X_1.loc[:, 'year'] = date.dt.year
X_1.loc[:, 'month'] = date.dt.month
X_1.loc[:, 'week'] = date.dt.isocalendar().week.astype(np.int32)
X_1.loc[:, 'day'] = date.dt.weekday

In [ ]:
date = pd.to_datetime(X_2['date'], format='%Y%m%d')
X_2.loc[:, 'year'] = date.dt.year
X_2.loc[:, 'month'] = date.dt.month
X_2.loc[:, 'week'] = date.dt.isocalendar().week.astype(np.int32)
X_2.loc[:, 'day'] = date.dt.weekday

In [ ]:
X_1 = X_1.drop(columns='date')
X_2 = X_2.drop(columns='date')

## Train_test_split

In [ ]:
X_train_01, X_test_01, y_train_01, y_test_01 = train_test_split(X_1, Y['outtrn_cumsum'], test_size=0.1, random_state=42)
X_train_02, X_test_02, y_train_02, y_test_02 = train_test_split(X_2, Y['HeatingEnergyUsage_cumsum'], test_size=0.1, random_state=42)

In [ ]:
X_train_01 = X_train_01.reset_index(drop=True)
X_test_01 = X_test_01.reset_index(drop=True)
y_train_01 = y_train_01.reset_index(drop=True)
y_test_01 = y_test_01.reset_index(drop=True)

In [ ]:
X_train_02 = X_train_02.reset_index(drop=True)
X_test_02 = X_test_02.reset_index(drop=True)
y_train_02 = y_train_02.reset_index(drop=True)
y_test_02 = y_test_02.reset_index(drop=True)

## Modeling

## AutoML

In [ ]:
automl = AutoML(mode="Optuna",
                eval_metric="rmse",
                algorithms=["LightGBM"],
                optuna_time_budget=3600,   # tune each algorithm for 30 minutes
                total_time_limit=48*3600,  # total time limit, set large enough to have time to compute all steps
                features_selection=False,  # switch off feature engineering
                mix_encoding=False,
                golden_features=False,
                kmeans_features=False,
                random_state=42)
automl.fit(X_train_01,y_train_01)

AutoML directory: AutoML_2
Expected computing time:
Time for tuning with Optuna: len(algorithms) * optuna_time_budget = 3600 seconds
There is no time limit for ML model training after Optuna tuning (total_time_limit parameter is ignored).
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['LightGBM']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['simple_algorithms', 'default_algorithms', 'ensemble', 'stack', 'ensemble_stacked']
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 1 model


[I 2023-09-16 22:50:12,119] A new study created in memory with name: no-name-85d4d969-c5da-4159-860a-9578a45badef
[W 2023-09-16 22:50:12,167] Trial 0 failed with parameters: {'learning_rate': 0.025, 'num_leaves': 321, 'lambda_l1': 2.5348407664333426e-07, 'lambda_l2': 3.3323645788192616e-08, 'feature_fraction': 0.9063233107042159, 'bagging_fraction': 0.7207805142313962, 'bagging_freq': 5, 'min_data_in_leaf': 3, 'extra_trees': True} because of the following error: The value None could not be cast to float..
[W 2023-09-16 22:50:12,170] Trial 0 failed with value None.
[W 2023-09-16 22:50:12,175] Trial 1 failed with parameters: {'learning_rate': 0.1, 'num_leaves': 1076, 'lambda_l1': 7.71800699380605e-05, 'lambda_l2': 4.17890272377219e-06, 'feature_fraction': 0.7282970324241945, 'bagging_fraction': 0.3976457038513679, 'bagging_freq': 3, 'min_data_in_leaf': 37, 'extra_trees': False} because of the following error: The value None could not be cast to float..
[W 2023-09-16 22:50:12,179] Trial 1

Optuna optimizes LightGBM with time budget 3600 seconds eval_metric rmse (minimize)
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'


[W 2023-09-16 22:50:12,317] Trial 9 failed with parameters: {'learning_rate': 0.025, 'num_leaves': 1836, 'lambda_l1': 7.278322158604965e-06, 'lambda_l2': 9.782893389157736e-08, 'feature_fraction': 0.45955461605871073, 'bagging_fraction': 0.5989754563094573, 'bagging_freq': 6, 'min_data_in_leaf': 87, 'extra_trees': False} because of the following error: The value None could not be cast to float..
[W 2023-09-16 22:50:12,322] Trial 9 failed with value None.
[W 2023-09-16 22:50:12,335] Trial 10 failed with parameters: {'learning_rate': 0.0125, 'num_leaves': 1932, 'lambda_l1': 8.106369361161699e-06, 'lambda_l2': 0.0004667836070663388, 'feature_fraction': 0.792113278256814, 'bagging_fraction': 0.5545407253018018, 'bagging_freq': 7, 'min_data_in_leaf': 97, 'extra_trees': False} because of the following error: The value None could not be cast to float..
[W 2023-09-16 22:50:12,340] Trial 10 failed with value None.
[W 2023-09-16 22:50:12,354] Trial 11 failed with parameters: {'learning_rate': 0.

Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'
Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'


...

[W 2023-09-16 22:51:38,019] Trial 4999 failed with parameters: {'learning_rate': 0.0125, 'num_leaves': 2006, 'lambda_l1': 0.005805743129710636, 'lambda_l2': 3.5554152440514917e-06, 'feature_fraction': 0.963002213927409, 'bagging_fraction': 0.5720071610464634, 'bagging_freq': 3, 'min_data_in_leaf': 62, 'extra_trees': True} because of the following error: The value None could not be cast to float..
[W 2023-09-16 22:51:38,021] Trial 4999 failed with value None.


Exception in LightgbmObjective train() got an unexpected keyword argument 'verbose_eval'


[W 2023-09-16 22:51:39,797] There are no complete trials.
[W 2023-09-16 22:51:40,187] Your study does not have any completed trials.
[W 2023-09-16 22:51:40,469] Study instance does not contain completed trials.
ERROR:supervised.exceptions:No models produced. 
Please check your data or submit a Github issue at https://github.com/mljar/mljar-supervised/issues/new.


There was an error during 1_Optuna_LightGBM training.
Please check AutoML_2/errors.md for details.


AutoMLException: ignored

In [ ]:
y_pred_01 = automl.predict(X_test_01)

In [ ]:
automl = AutoML(mode="Compete", eval_metric='rmse', algorithms=['LightGBM', 'CatBoost', 'Xgboost'], features_selection = False, random_state = 42)
automl.fit(X_train_02, y_train_02)

AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['LightGBM', 'CatBoost', 'Xgboost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 194164.417731 trained in 1.77 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 3 models
1_Default_LightGBM rmse 47814.599674 trained in 81.55 seconds
2_Default_Xgboost rmse 51270.771348 trained in 107.61 seconds
3_Default_CatBoost rmse 49665.731632 trained in 227.36 seconds
* Step not_so_random will try to check up to 27 models
13_L

AutoML(algorithms=['LightGBM', 'CatBoost', 'Xgboost'], eval_metric='rmse',
       features_selection=False, mode='Compete', random_state=42)

In [ ]:
y_pred_02 = automl.predict(X_test_02)

In [ ]:
y_test = pd.concat([y_test_01, y_test_02], axis=1)

In [ ]:
y_pred_concatenated = np.column_stack((y_pred_01, y_pred_02))
y_pred = pd.DataFrame(y_pred_concatenated, columns=['y_pred_01', 'y_pred_02'])

In [ ]:
# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_test, y_pred)


rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

In [ ]:
### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 27495.44231170572
R2_score: 0.9966474831969616
